<a href="https://colab.research.google.com/github/NaveenShetter1/deep-learning-notebooks/blob/main/adas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Object Detection with YOLO

In [2]:
!wget https://raw.githubusercontent.com/ultralytics/yolov5/master/data/coco.names


--2024-11-12 12:41:07--  https://raw.githubusercontent.com/ultralytics/yolov5/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-11-12 12:41:07 ERROR 404: Not Found.



In [3]:
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget https://raw.githubusercontent.com/ultralytics/yolov5/master/data/coco.names

--2024-11-12 12:42:00--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  12.4MB/s    in 43s     

2024-11-12 12:43:16 (5.54 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]

--2024-11-12 12:43:16--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg.1’

yolov3.cfg.1        100%[===================>]   8.15K  --.-KB/s    in 0s   

In [10]:
import cv2
import numpy as np

# Upload yolov3.weights, yolov3.cfg, and coco.names files before running

# Load YOLO model (Pre-trained weights and config)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = net.getUnconnectedOutLayersNames()


# Load class labels for YOLO (COCO dataset)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Initialize the video stream (use a camera or video file)
cap = cv2.VideoCapture(0)  # Replace '0' with the path to your video file

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the image for YOLO (resize and normalize)
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Post-process YOLO outputs
    class_ids = []
    confidences = []
    boxes = []
    height, width, channels = frame.shape

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Threshold for object detection
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maxima suppression to avoid multiple boxes around the same object
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw detected objects
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)  # Green color for bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Show the frame
    cv2.imshow("ADAS Object Detection", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [5]:
# Lane Detection (Hough Transform)

In [11]:
import cv2
import numpy as np

def detect_lane(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Define a region of interest (ROI) where lanes are expected
    height, width = edges.shape
    roi_vertices = [(0, height), (width // 2, int(height * 0.6)), (width, height)]
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, np.array([roi_vertices], dtype=np.int32), 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Detect lines using Hough Transform
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=50)

    # Draw the detected lanes on the original image
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    return image

# Initialize the video stream (use a camera or video file)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect and draw lane markings
    lane_frame = detect_lane(frame)

    # Show the frame with lane markings
    cv2.imshow("ADAS Lane Detection", lane_frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
# Integration for Real-Time ADAS

In [12]:
import cv2
import numpy as np

# Object detection and lane detection functions from above
# Use the object detection function and lane detection function

def process_frame(frame):
    # Step 1: Detect lanes
    lane_frame = detect_lane(frame)

    # Step 2: Detect objects (e.g., vehicles, pedestrians)
    # Object detection code from the YOLO model goes here

    # Combine results
    combined_frame = lane_frame  # Add object detection on top of lane detection
    return combined_frame

# Initialize the video stream (use a camera or video file)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with both lane and object detection
    output_frame = process_frame(frame)

    # Show the output frame
    cv2.imshow("ADAS Real-Time System", output_frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
